In [1]:
%load_ext autoreload
%autoreload 2
import torch
from deepy.nn import Tanh, Relu, Linear, Sequential, Module
from deepy.loss import MSE
from deepy.optim import SGD
from deepy.tensor import Variable
from deepy.utils import graph_repr

In [2]:
import dlc_practical_prologue as prologue
from tqdm import tqdm

In [3]:
train_input, train_target, test_input, test_target = prologue.load_data(one_hot_labels=True, normalize=True)
gamma = 0.9
train_target *= gamma 

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [4]:
# define network
l1 = Linear(784, 50)
t1 = Tanh()
l2 = Linear(50, 10)
t2 = Tanh()
net = Sequential([l1, t1, l2, t2])
dummy_inputs = Variable(train_input[0])
print(graph_repr(net, dummy_inputs))

X => Linear(784, 50) => Tanh() => Linear(50, 10) => Tanh() => y


In [5]:
class Net(Module):
    def __init__(self):
        self.l1 = Linear(784, 50)
        self.a1 = Tanh()
        self.l2 = Linear(50, 10)
        self.a2 = Relu()
        self.tes = Variable(torch.empty(2,2), requires_grad=True)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.a1(out)
        out = self.l2(out)
        out = self.a2(out)
        return out

net = Net()

dummy_inputs = Variable(train_input[0])
print(graph_repr(net, dummy_inputs))
net.param()

X => Linear(784, 50) => Tanh() => Linear(50, 10) => Relu() => y


In [6]:
optimizer = SGD(net.param(), lr=0.1 / train_input.shape[0])  # TODO if time add weight regularization 
criterion = MSE()
optimizer.zero_grad()

In [7]:
from deepy.tensor import Variable
for i in tqdm(range(100)):
    optimizer.zero_grad()
    for x, t in zip(train_input, train_target):
        x = Variable(x)
        out = net(x)
        loss = criterion(out, t)
        loss.backward()
    optimizer.step()

100%|██████████| 100/100 [00:30<00:00,  2.64it/s]


In [8]:
nb_correct = 0
for x, t in zip(test_input, test_target):
    pred = net(x)
    pred = pred.data
    if pred.argmax() == t.argmax():
        nb_correct+=1
print(f"{nb_correct} class predicted correctly")

786 class predicted correctly
